# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,massaguet,12.4755,15.4365,37.01,12,4,4.80,TD,1698680000
1,1,enewetak,11.3474,162.3373,29.12,73,83,7.62,MH,1698680001
2,2,berdigestyakh,62.0989,126.6992,-23.95,100,100,0.90,RU,1698680001
3,3,masterton,-40.9597,175.6575,13.42,99,100,2.01,NZ,1698680002
4,4,nadi,-17.8000,177.4167,24.00,88,20,1.54,FJ,1698680002


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=650,
    frame_height=500,
    size="Humidity",
    color="City"
    )
# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_condition = city_data_df.loc[(city_data_df['Max Temp'] >=25) & (city_data_df['Max Temp']<=31) 
& (city_data_df['Wind Speed'] <=5) & (city_data_df['Cloudiness'] <10)]

# Drop any rows with null values
city_cleaned = ideal_condition.dropna()

# Display sample data
city_cleaned

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
75,75,salalah,17.0151,54.0924,28.05,69,0,2.06,OM,1698680062
89,89,howrah,22.5892,88.3103,27.00,69,3,1.54,IN,1698680071
96,96,berbera,10.4396,45.0143,28.76,73,7,4.98,SO,1698680075
165,165,saint-pierre,-21.3393,55.4781,25.82,73,0,2.06,RE,1698680129
194,194,cabo san lucas,22.8909,-109.9124,28.20,71,0,2.68,MX,1698679869
251,251,arraial do cabo,-22.9661,-42.0278,30.98,66,0,3.60,BR,1698680187
301,301,porbandar,21.6422,69.6093,28.49,54,0,4.56,IN,1698680220
318,318,veraval,20.9000,70.3667,27.57,57,0,0.98,IN,1698680234
419,419,rengasdengklok,-6.1592,107.2981,30.07,66,5,1.03,ID,1698680305
436,436,kayangel,8.0823,134.7173,28.39,76,9,3.27,PW,1698680321


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
#%%capture --no-display
#used capture to remove false error displaying an empty column

# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_cleaned[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
75,salalah,OM,17.0151,54.0924,69,
89,howrah,IN,22.5892,88.3103,69,
96,berbera,SO,10.4396,45.0143,73,
165,saint-pierre,RE,-21.3393,55.4781,73,
194,cabo san lucas,MX,22.8909,-109.9124,71,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 1
params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key     
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = hotel_df.loc[index, "Lng"] 
    lat = hotel_df.loc[index, "Lat"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
salalah - nearest hotel: Muscat International Hotel
howrah - nearest hotel: Sun India Guest House
berbera - nearest hotel: Al Madina Hotel
saint-pierre - nearest hotel: Tropic Hotel
cabo san lucas - nearest hotel: Comfort Rooms
arraial do cabo - nearest hotel: No hotel found
porbandar - nearest hotel: Toran Tourist Bungalow
veraval - nearest hotel: Shyam
rengasdengklok - nearest hotel: No hotel found
kayangel - nearest hotel: No hotel found
at taj - nearest hotel: No hotel found
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
celestun - nearest hotel: Hotel Flamingo Playa
grand baie - nearest hotel: Safari
khark - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
75,salalah,OM,17.0151,54.0924,69,Muscat International Hotel
89,howrah,IN,22.5892,88.3103,69,Sun India Guest House
96,berbera,SO,10.4396,45.0143,73,Al Madina Hotel
165,saint-pierre,RE,-21.3393,55.4781,73,Tropic Hotel
194,cabo san lucas,MX,22.8909,-109.9124,71,Comfort Rooms
251,arraial do cabo,BR,-22.9661,-42.0278,66,No hotel found
301,porbandar,IN,21.6422,69.6093,54,Toran Tourist Bungalow
318,veraval,IN,20.9000,70.3667,57,Shyam
419,rengasdengklok,ID,-6.1592,107.2981,66,No hotel found
436,kayangel,PW,8.0823,134.7173,76,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=550,
    size="Humidity",
    color="City",
    hover_cols=["Country","Hotel Name"]
    )

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)